# Data loading
We start again as usuall by loading all the data from the successful and from the unsuccessful companies.

1. We import relevant libraries

In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from functools import reduce
import math
import urllib
import difflib

Then we load the data:
1. load the excel files.

In [15]:
'''
First we load both spreadsheets and extra all the names of the workbooks
'''

xls_Success_Comp       = pd.ExcelFile('data/success/Moneyball 1.1_ Success.xlsx')
xls_Success_Comp_Names = xls_Success_Comp.sheet_names


xls_Unsuccess_Comp = pd.ExcelFile('data/success/Moneyball 1.1_ Success.xlsx')
xls_Unsuccess_Comp_Sheet_Names = xls_Unsuccess_Comp.sheet_names


'Merging all of the successful data into one dataframe and creating a column called "success_flag" \nwhich will denote if the company is successful or not '

2. Load each sheet into a dataframe.

In [37]:
'''
Importing all of the excel sheets of successful informaiton into a list of dataframes
'''
df_list_SuccessComp = [pd.read_excel(r'data/success/Moneyball 1.1_ Success.xlsx',
                                         sheet_name= sheetname) for sheetname in xls_Success_Comp_Names]
                                         
# df_SuccessfulCompanies[1].drop(index = [503, 504], inplace=True)
# df_SuccessfulCompanies[2].drop(index = [573, 574], inplace=True)
# df_SuccessfulCompanies[3].drop(index = [522, 523], inplace=True)

'''
Importing all of the excel sheets of unsuccessful informaiton, into a list of dataframes

'''
df_list_UnsuccessComp = [pd.read_excel(r'data/success/Moneyball 1.1_ Success.xlsx',
                                         sheet_name= sheetname) for sheetname in xls_Unsuccess_Comp_Sheet_Names]


In [38]:
# lets do some sanity checks
print(len(df_list_SuccessComp))

# df_list_SuccessfulCompanies[0].head()
df_list_SuccessComp[3].head()
df_list_SuccessComp[5].head()


6


,README
0,Using the 'investors_uuids' in the 'Funding Ro...
1,https://drive.google.com/file/d/1wcMns7E_cAD0R...


We can see that the Excel spreadsheet contains two sheets with only links to the CSV files of founder linkedin profiles and investor profiles, we have downloaded those and to get started we will not concern ourselves with the data from these two spreadsheets. I will drop them for now.

In [39]:
indexes = [3, 5]
for index in sorted(indexes, reverse=True):
    del df_list_SuccessComp[index]
    del df_list_UnsuccessComp[index]

len(df_list_SuccessComp), len(df_list_UnsuccessComp)

(4, 4)

3. Join all of the dataframes together.

In [44]:
'''Merging all of the successful data into one dataframe and creating a column called "success_flag" 
which will denote if the company is successful or not '''
df_SuccessfulCompanies_merged = reduce(lambda  left,right: pd.merge(left, right, on=['org_uuid'],
                                                                    how='outer', 
                                                                    suffixes=('', '_y')),
                                                                    df_list_SuccessComp)
df_SuccessfulCompanies_merged['success_flag']  = 1

# drop duplicate columns
df_SuccessfulCompanies_merged.drop(df_SuccessfulCompanies_merged.filter(regex='_y$').columns, axis=1, inplace=True)

'''Merging all of the unsuccessful data into one dataframe and creating a column called "success_flag" 
which will denote if the company is successful or not '''


df_UnsuccessfulCompanies_merged = reduce(lambda  left,right: pd.merge(left, right, on=['org_uuid'],
                                                                      how='outer', 
                                                                      suffixes=('', '_y')),
                                                                      df_list_UnsuccessComp)
df_UnsuccessfulCompanies_merged['success_flag']  = 0

# drop duplicate columns
df_UnsuccessfulCompanies_merged.drop(df_UnsuccessfulCompanies_merged.filter(regex='_y$').columns, axis=1, inplace=True)

'''Merge the successful and unsuccessful data into one datafram'''
df_All_Companies_merged = pd.concat([df_SuccessfulCompanies_merged, df_UnsuccessfulCompanies_merged], ignore_index=True, sort=False)

In [46]:
# sanity checks

df_SuccessfulCompanies_merged.head()

,org_uuid,org_name,domain,status,founded_on,category_list,category_groups_list,country_code,city,short_description,long_description,founder_linkedin_url,investment_type,round_created_at,raised_amount_usd,investor_count,investors_names,investors_uuids,success_flag
0,a2323b6c-29b9-4750-905c-cdcbd9ce92b8,10X Banking,10xbanking.com,operating,2016-01-01,"Banking,Financial Services,FinTech,Software","Financial Services,Lending and Investments,Sof...",GBR,London,10X Banking is a financial service technology ...,"10x Banking (10x), a financial services techno...",https://www.linkedin.com/in/antony-jenkins/,series_a,2017-09-21 01:07:18.000000 UTC,45867909.0,2.0,"Oliver Wyman,Ping An Global Voyager","88213ccc-71f8-1024-72cf-44e863af4993,c8cb5641-...",1
1,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,10xgenomics.com,ipo,2012-01-01,"Biotechnology,Genetics,Health Care,Medical","Biotechnology,Health Care,Science and Engineering",USA,Pleasanton,10X Genomics ELV solutions and security system...,10x Genomics is creating revolutionary DNA seq...,https://www.linkedin.com/in/ben-hindson-09739813/,seed,2018-04-27 04:18:33.000000 UTC,4700000.0,1.0,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,1
2,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,10xgenomics.com,ipo,2012-01-01,"Biotechnology,Genetics,Health Care,Medical","Biotechnology,Health Care,Science and Engineering",USA,Pleasanton,10X Genomics ELV solutions and security system...,10x Genomics is creating revolutionary DNA seq...,https://www.linkedin.com/in/serge-saxonov-7b42...,seed,2018-04-27 04:18:33.000000 UTC,4700000.0,1.0,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,1
3,0680987f-23cd-4fa5-850c-38326fffaa36,"111, Inc.",corporate.111.com.cn,ipo,2010-01-01,"Health Care,Medical",Health Care,CHN,Shanghai,"111, Inc. operates as a digital and mobile hea...","111,Inc. is a leading digital and mobile healt...",https://www.linkedin.com/in/junling-liu-2b90724/,series_a,2018-09-14 07:49:39.000000 UTC,2100000.0,1.0,Ivy Capital,f8eed790-8c07-4e49-c873-0f7245e393ff,1
4,a79a6ae8-3c74-0db7-7e6b-b4f7662d37e8,17zuoye,ucenter.17zuoye.com,ipo,2011-01-01,"E-Learning,EdTech,Education,Internet,Language ...","Education,Internet Services,Software",CHN,Shanghai,17zuoye is an online learning platform for K-1...,17zuoye is an online learning platform for K-1...,https://www.linkedin.com/in/dun-xiao-3622629/,series_a,2015-02-20 15:29:12.000000 UTC,5000000.0,1.0,Shunwei Capital,24540ba2-35b0-a757-f75e-87565c50692e,1
